In [24]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf      # sf = spark functions
import pyspark.sql.types as st          # st = spark types


sparkql = pyspark.sql.SparkSession.builder.master('local').getOrCreate()

import os
data_dir = '../data'

file_names = ['AAPL','ADBE','AMZN', 'CRM', 'CSCO', 'GOOGL', 'IBM','INTC','META','MSFT','NFLX','NVDA','ORCL','TSLA']

columns = 'stock_name string, date date, open float, high float, low float, close float, adj_close float, volume int'

df = sparkql.createDataFrame(data=[],schema=columns)

for csv in file_names:
    idf = sparkql.read.csv(os.path.join(data_dir,csv+'.csv'), header=True)
    idf = idf.withColumn('stock_name', sf.lit(csv))
    idf = idf.toDF('date', 'open', 'high', 'low', 'close', 'adj_close', 'volume', 'stock_name')
    #df.select('stock_name', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume')
    idf = idf.select('stock_name', 'date', 'open', 'high', 'low', 'close', 'adj_close', 'volume')
    df.unionByName(idf)

df.show()



+----------+----+----+----+---+-----+---------+------+
|stock_name|date|open|high|low|close|adj_close|volume|
+----------+----+----+----+---+-----+---------+------+
+----------+----+----+----+---+-----+---------+------+

